In [ ]:
!pip install pycorrelate

In [231]:
from scipy.io import wavfile
from scipy import signal as sig
from scipy.linalg import toeplitz
import numpy as np
import pycorrelate as pyc

In [232]:
Fs, signal = wavfile.read('Abr.wav')

In [233]:
signal_len = len(signal)
signal_len

10812144

In [234]:
signal = np.array(signal[0:14000, 1])
L = len(signal)

In [235]:
signal = signal/max(signal)

In [236]:
var = 0.1
noise = np.sqrt(var)*np.random.normal(size=L)

In [237]:
noisySig = signal + noise;

In [238]:
wavfile.write('noisy_Abr.wav', Fs, noisySig)

In [239]:
n = 100

In [240]:
p = np.zeros((n, 1))
p[0] = var
#p = np.asmatrix(p)

In [241]:
R = sig.correlate(noise, noise)

In [242]:
R = R[n:2*n]

In [243]:
R = toeplitz(R)

In [244]:
R

array([[-0.19043675, -1.13764981, -0.27253211, ...,  0.71147154,
         1.71289901,  1.57656616],
       [-1.13764981, -0.19043675, -1.13764981, ...,  0.99981925,
         0.71147154,  1.71289901],
       [-0.27253211, -1.13764981, -0.19043675, ...,  0.27323497,
         0.99981925,  0.71147154],
       ...,
       [ 0.71147154,  0.99981925,  0.27323497, ..., -0.19043675,
        -1.13764981, -0.27253211],
       [ 1.71289901,  0.71147154,  0.99981925, ..., -1.13764981,
        -0.19043675, -1.13764981],
       [ 1.57656616,  1.71289901,  0.71147154, ..., -0.27253211,
        -1.13764981, -0.19043675]])

In [245]:
w0 = np.matmul((np.linalg.inv(R)), p)
#w0 = np.asmatrix(w0)

In [246]:
w0 = w0.reshape(n)

In [247]:
error = np.zeros((L, 1))
#error = np.asmatrix(error)

s = np.append([0], noise)
#s = np.asmatrix(s)

w1 = -1*np.ones((n, 1))
#w1 = np.asmatrix(w1)

m = 0.001

wt = np.zeros((n, L))
#wt = np.asmatrix(wt)
wt[:,0] = w0

In [202]:
w1 = np.float64(w1)

In [248]:
for i in np.arange(n, L,1):
  w1 = w1 + m*(p- np.matmul(R,w1))
  wt[:, i] = w1.reshape(n)
  temp_s = s[i-(n-1):i+1][::-1]
  error[i] = np.dot(temp_s, w0)

In [249]:
e = error

In [250]:
e = np.squeeze(np.asarray(e[1:L]))

In [251]:
e = np.append(e,0)

In [252]:
denoised = noisySig - e

In [253]:
wavfile.write('denoised_Abr.wav', Fs, denoised)

In [254]:
SNR_noisy = np.sum(np.square(signal))/np.sum(np.square(noise))

In [256]:
SNR_noisy

0.26571320992526054

In [257]:
remainedNoise = denoised - signal

In [258]:
SNR_denoised = np.sum(np.square(signal))/np.sum(np.square(remainedNoise))

In [259]:
SNR_denoised

0.2679141987559448